# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2022 11:53AM,239556,10,MET211212,"Methapharm, Inc.",Released
1,Jun 14 2022 11:53AM,239556,10,MET211214,"Methapharm, Inc.",Released
2,Jun 14 2022 11:53AM,239556,10,MET211215,"Methapharm, Inc.",Released
3,Jun 14 2022 11:52AM,239555,10,MET211176,"Methapharm, Inc.",Released
4,Jun 14 2022 11:52AM,239555,10,MET211182,"Methapharm, Inc.",Released
5,Jun 14 2022 11:52AM,239555,10,MET211192,"Methapharm, Inc.",Released
6,Jun 14 2022 11:52AM,239554,10,8142483,Eywa Pharma Inc.,Released
7,Jun 14 2022 11:52AM,239554,10,8143020-PR,Eywa Pharma Inc.,Released
8,Jun 14 2022 11:52AM,239554,10,8143026,Eywa Pharma Inc.,Released
9,Jun 14 2022 11:52AM,239554,10,8143027,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,239548,Released,47
37,239553,Released,6
38,239554,Released,11
39,239555,Released,3
40,239556,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239548,NaN,NaN,47.0
239553,NaN,NaN,6.0
239554,NaN,NaN,11.0
239555,NaN,NaN,3.0
239556,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,0.0,15.0,139.0
239463,26.0,0.0,3.0
239479,0.0,0.0,33.0
239481,0.0,0.0,1.0
239482,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,0,15,139
239463,26,0,3
239479,0,0,33
239481,0,0,1
239482,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,0,15,139
1,239463,26,0,3
2,239479,0,0,33
3,239481,0,0,1
4,239482,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,,15,139
1,239463,26,,3
2,239479,,,33
3,239481,,,1
4,239482,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc."
3,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc."
6,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.
17,Jun 14 2022 11:48AM,239553,10,Emerginnova
23,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc."
177,Jun 14 2022 11:19AM,239548,10,ISDIN Corporation
224,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation
225,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation
226,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc."
227,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc.",,,3
1,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc.",,,3
2,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.,,,11
3,Jun 14 2022 11:48AM,239553,10,Emerginnova,,,6
4,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc.",,15,139
5,Jun 14 2022 11:19AM,239548,10,ISDIN Corporation,,,47
6,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation,,,1
7,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation,,,1
8,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc.",,,1
9,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc.",,1,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc.",3,,
1,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc.",3,,
2,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.,11,,
3,Jun 14 2022 11:48AM,239553,10,Emerginnova,6,,
4,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc.",139,15,
5,Jun 14 2022 11:19AM,239548,10,ISDIN Corporation,47,,
6,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation,1,,
7,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation,1,,
8,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc.",1,,
9,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc.",1,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc.",3,,
1,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc.",3,,
2,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.,11,,
3,Jun 14 2022 11:48AM,239553,10,Emerginnova,6,,
4,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc.",139,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc.",3,NaN,NaN
1,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc.",3,NaN,NaN
2,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.,11,NaN,NaN
3,Jun 14 2022 11:48AM,239553,10,Emerginnova,6,NaN,NaN
4,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc.",139,15.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc.",3,0.0,0.0
1,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc.",3,0.0,0.0
2,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.,11,0.0,0.0
3,Jun 14 2022 11:48AM,239553,10,Emerginnova,6,0.0,0.0
4,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc.",139,15.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5030150,297,8.0,2.0
12,239544,1,1.0,0.0
15,958043,204,16.0,0.0
19,239490,1,0.0,0.0
20,478998,25,8.0,26.0
21,718448,3,0.0,0.0
22,239497,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5030150,297,8.0,2.0
1,12,239544,1,1.0,0.0
2,15,958043,204,16.0,0.0
3,19,239490,1,0.0,0.0
4,20,478998,25,8.0,26.0
5,21,718448,3,0.0,0.0
6,22,239497,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,297,8.0,2.0
1,12,1,1.0,0.0
2,15,204,16.0,0.0
3,19,1,0.0,0.0
4,20,25,8.0,26.0
5,21,3,0.0,0.0
6,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,297.0
1,12,Released,1.0
2,15,Released,204.0
3,19,Released,1.0
4,20,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,26.0,0.0,0.0
Executing,8.0,1.0,16.0,0.0,8.0,0.0,0.0
Released,297.0,1.0,204.0,1.0,25.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,26.0,0.0,0.0
1,Executing,8.0,1.0,16.0,0.0,8.0,0.0,0.0
2,Released,297.0,1.0,204.0,1.0,25.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,26.0,0.0,0.0
1,Executing,8.0,1.0,16.0,0.0,8.0,0.0,0.0
2,Released,297.0,1.0,204.0,1.0,25.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()